## Deep Learning Project – Handwritten Digit Recognition using Keras

#### Prerequisites


In [2]:
#pip install keras

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)



In [4]:
#pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install tensorflow

#### The MNIST dataset

This is probably one of the most popular datasets among machine learning and deep learning enthusiasts. The [MNIST dataset](http://yann.lecun.com/exdb/mnist/) contains 60,000 training images of handwritten digits from zero to nine and 10,000 images for testing. So, the MNIST dataset has 10 different classes. The handwritten digits images are represented as a 28×28 matrix where each cell contains grayscale pixel value.

#### Import the libraries and load the dataset

In [3]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [7]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [8]:
x_train.shape, y_train.shape

((60000, 28, 28), (60000,))

#### Preprocess the data

In [11]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
num_classes = 10
input_shape = (28, 28, 1)

In [12]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [13]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


#### Create the model

In [14]:
batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

#### Train the model

In [15]:
# fitting the model using model.fit()
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 150s 315ms/step - loss: 2.2874 - accuracy: 0.1455 - val_loss: 2.2423 - val_accuracy: 0.4206
Epoch 2/10
469/469 [==============================] - 137s 291ms/step - loss: 2.2357 - accuracy: 0.2846 - val_loss: 2.1720 - val_accuracy: 0.6032
Epoch 3/10
469/469 [==============================] - 129s 276ms/step - loss: 2.1641 - accuracy: 0.4051 - val_loss: 2.0684 - val_accuracy: 0.6608
Epoch 4/10
469/469 [==============================] - 126s 268ms/step - loss: 2.0595 - accuracy: 0.5005 - val_loss: 1.9156 - val_accuracy: 0.7040
Epoch 5/10
469/469 [==============================] - 133s 283ms/step - loss: 1.9065 - accuracy: 0.5717 - val_loss: 1.7047 - val_accuracy: 0.7432
Epoch 6/10
469/469 [==============================] - 130s 277ms/step - loss: 1.7009 - accuracy: 0.6260 - val_loss: 1.4485 - val_accuracy: 0.7723
Epoch 7/10
469/469 [==============================] - 132s 281ms/step - loss: 1.4782 - accuracy: 0.6545 - val_loss: 1.1923 -

In [16]:
# saving the model in "mnist.h5"
model.save('mnist.h5')

#### Evaluate the model

In [17]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7182334065437317
Test accuracy: 0.8431000113487244


#### GUI to predict digits

In [21]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

KeyboardInterrupt: 